# Zadania z kolokwium 2019/20
wiele rzeczy może być źle

## Zadania #1
**1.1 Wypisz wszystkich członków biblioteki z adresami i info czy jest dzieckiem czy nie i
ilość wypożyczeń w poszczególnych latach i miesiącach.**

In [ ]:
USE Library

In [ ]:
-- to może być źle, nie wiem czy uwzględniać loan

SELECT 
    M.firstname, M.lastname,
    CASE
        WHEN A1.street IS NOT NULL THEN A1.street + ' ' + A1.city + ' ' + A1.state
        ELSE A2.street + ' ' + A2.city + ' ' + A2.state
    END AS address,
    CASE
        WHEN J.member_no IS NOT NULL THEN 'child'
        ELSE 'adult'
    END AS child_or_adult,
    YEAR(LH.out_date) AS year,
    MONTH(LH.out_date) AS MONTH,
    COUNT(LH.isbn) AS borrowed_books
FROM member AS M
LEFT OUTER JOIN juvenile AS J
ON M.member_no = J.member_no
LEFT OUTER JOIN adult AS A1
ON M.member_no = A1.member_no
LEFT OUTER JOIN adult AS A2
ON J.adult_member_no = A2.member_no
LEFT OUTER JOIN loanhist AS LH
ON M.member_no = LH.member_no
WHERE LH.out_date IS NOT NULL
GROUP BY  M.firstname, M.lastname, A1.street, A1.city, A1.state, A2.street, A2.city, A2.state, J.member_no, LH.out_date

**1.2 Zamówienia z Freight większym niż AVG danego roku.**

In [ ]:
USE Northwind

In [ ]:
SELECT OrderID
FROM Orders AS O
WHERE Freight > 
    (SELECT AVG(Freight) 
    FROM Orders AS O2 
    WHERE YEAR(ShippedDate) = YEAR(O.ShippedDate))

**1.3 Klienci, którzy nie zamówili nigdy nic z kategorii 'Seafood' w trzech wersjach.**

In [ ]:
SELECT CompanyName
FROM Customers
WHERE Customers.CustomerID NOT IN
    (SELECT C.CustomerID
    FROM Customers AS C
    INNER JOIN Orders AS O
    ON C.CustomerID = O.CustomerID
    INNER JOIN [Order Details] AS OD
    ON OD.OrderID = O.OrderID
    INNER JOIN Products AS P
    ON OD.ProductID = P.ProductID
    INNER JOIN Categories AS CA
    ON P.CategoryID = CA.CategoryID
    WHERE CA.CategoryName = 'Seafood')

In [ ]:
SELECT CompanyName
FROM Customers
LEFT OUTER JOIN 
    (Orders AS O
    INNER JOIN [Order Details] AS OD
    ON O.OrderID = OD.OrderID
    INNER JOIN Products AS P
    ON OD.ProductID = P.ProductID
    INNER JOIN Categories AS C
    ON C.CategoryId = P.CategoryID AND C.CategoryName = 'Seafood')
ON Customers.CustomerID = O.CustomerID
WHERE C.CategoryID IS NULL

**1.4 Dla każdego klienta najczęściej zamawianą kategorię w dwóch wersjach.**

In [ ]:
SELECT C2.CompanyName, 
    (SELECT TOP 1 CA.CategoryName
    FROM Customers AS C
    INNER JOIN Orders AS O
    ON C.CustomerID = O.CustomerID
    INNER JOIN [Order Details] AS OD
    ON OD.OrderID = O.OrderID
    INNER JOIN Products AS P
    ON P.ProductID = OD.ProductID
    INNER JOIN Categories AS CA
    ON CA.CategoryID = P.CategoryID
    WHERE C.CustomerID = C2.CustomerID
    GROUP BY CA.CategoryName
    ORDER BY COUNT(*) DESC) AS MostBoughtCat
FROM Customers AS C2

## Zadania #2
**2.1 Podział na company, year month i suma freight**

In [ ]:
SELECT C.CompanyName, YEAR(O.OrderDate) AS year, MONTH(O.OrderDate) AS month, SUM(O.Freight) AS TotalFreight
FROM Customers AS C
INNER JOIN Orders AS O
ON C.CustomerID = O.CustomerID
GROUP BY C.CompanyName, YEAR(O.OrderDate), MONTH(O.OrderDate)
ORDER BY 1

**2.2 Wypisać wszystkich czytelników, którzy nigdy nie wypożyczyli książki dane
adresowe i podział czy ta osoba jest dzieckiem (joiny, in, exists)**

In [ ]:
--źle, nie uwzględnia loan

SELECT
    M.firstname, M.lastname,
    CASE
        WHEN A1.street IS NOT NULL THEN A1.street + ' ' + A1.city + ' ' + A1.state
        ELSE A2.street + ' ' + A2.city + ' ' + A2.state
    END AS address,
    CASE
        WHEN J.member_no IS NOT NULL THEN 'child'
        ELSE 'adult'
    END AS child_or_adult
FROM member AS M
LEFT OUTER JOIN juvenile AS J
ON M.member_no = J.member_no
LEFT OUTER JOIN adult AS A1
ON M.member_no = A1.member_no
LEFT OUTER JOIN adult AS A2
ON J.adult_member_no = A2.member_no
LEFT OUTER JOIN loanhist AS LH
ON M.member_no = LH.member_no
WHERE LH.out_date IS NULL

In [ ]:
SELECT
    M.firstname, M.lastname,
    CASE
        WHEN A1.street IS NOT NULL THEN A1.street + ' ' + A1.city + ' ' + A1.state
        ELSE A2.street + ' ' + A2.city + ' ' + A2.state
    END AS address,
    CASE
        WHEN J.member_no IS NOT NULL THEN 'child'
        ELSE 'adult'
    END AS child_or_adult
FROM member AS M
LEFT OUTER JOIN juvenile AS J
ON M.member_no = J.member_no
LEFT OUTER JOIN adult AS A1
ON M.member_no = A1.member_no
LEFT OUTER JOIN adult AS A2
ON J.adult_member_no = A2.member_no
WHERE M.member_no NOT IN
    (SELECT M1.member_no
    FROM member AS M1
    INNER JOIN loanhist AS LH
    ON M1.member_no = LH.member_no)

In [ ]:
SELECT
    M.firstname, M.lastname,
    CASE
        WHEN A1.street IS NOT NULL THEN A1.street + ' ' + A1.city + ' ' + A1.state
        ELSE A2.street + ' ' + A2.city + ' ' + A2.state
    END AS address,
    CASE
        WHEN J.member_no IS NOT NULL THEN 'child'
        ELSE 'adult'
    END AS child_or_adult
FROM member AS M
LEFT OUTER JOIN juvenile AS J
ON M.member_no = J.member_no
LEFT OUTER JOIN adult AS A1
ON M.member_no = A1.member_no
LEFT OUTER JOIN adult AS A2
ON J.adult_member_no = A2.member_no
WHERE NOT EXISTS 
    (SELECT *
    FROM member AS M1
    INNER JOIN loanhist AS LH
    ON M1.member_no = LH.member_no AND M1.member_no = M.member_no)

**2.3 Najczęściej wybierana kategoria w 1997 dla każdego klienta**

In [ ]:
USE Northwind

In [ ]:
SELECT C2.CompanyName, 
    (SELECT TOP 1 CA.CategoryName
    FROM Customers AS C
    INNER JOIN Orders AS O
    ON C.CustomerID = O.CustomerID AND YEAR(O.OrderDate) = 1997
    INNER JOIN [Order Details] AS OD
    ON OD.OrderID = O.OrderID
    INNER JOIN Products AS P
    ON P.ProductID = OD.ProductID
    INNER JOIN Categories AS CA
    ON CA.CategoryID = P.CategoryID
    WHERE C.CustomerID = C2.CustomerID
    GROUP BY CA.CategoryName
    ORDER BY COUNT(*) DESC) AS MostBoughtCat
FROM Customers AS C2

**2.4 Dla każdego czytelnika imię nazwisko, suma książek wypożyczony przez tą osobę i
jej dzieci, który żyje w Arizona ma mieć więcej niż 2 dzieci lub kto żyje w Kalifornii
ma mieć więcej niż 3 dzieci**

In [ ]:
USE Library

In [ ]:
-- może być źle, nie uwzględnia loan

SELECT M.firstname, M.lastname, COUNT(*) + 
    (SELECT COUNT(*)
    FROM member AS M1
    INNER JOIN loanhist AS LH1
    ON M1.member_no = LH1.member_no
    WHERE M1.member_no = M.member_no)
FROM member AS M
INNER JOIN adult AS A
ON M.member_no = A.member_no
INNER JOIN juvenile AS J
ON A.member_no = J.adult_member_no
INNER JOIN loanhist AS LH
ON J.member_no = LH.member_no
GROUP BY M.member_no, M.firstname, M.lastname, A.state
HAVING (A.state = 'AZ' AND COUNT(J.member_no) > 2) OR (A.state = 'CA' AND COUNT(J.member_no) > 3)
-- nie wiem czy to działa w sumie, chyba nie

## Zadania #3
**3.1 Jaki był najpopularniejszy autor wśród dzieci w Arizonie w 2001**

In [ ]:
SELECT TOP 1 WITH TIES T.author
FROM title AS T
INNER JOIN loanhist AS LH
ON T.title_no = LH.title_no
INNER JOIN member AS M
ON M.member_no = LH.member_no
INNER JOIN juvenile AS J
ON J.member_no = M.member_no
INNER JOIN adult AS A
ON J.adult_member_no = A.member_no AND A.state = 'AZ'
GROUP BY T.author
ORDER BY T.author DESC

**3.2 Dla każdego dziecka wybierz jego imię nazwisko, adres, imię i nazwisko rodzica i
ilość książek, które oboje przeczytali w 2001**

In [ ]:
SELECT M.member_no, M.firstname, M.lastname, A.street, A.city, A.state, 
    MP.firstname as parentfirstname, MP.lastname as parentlastname,
    CASE
        WHEN M.member_no IN (SELECT member_no FROM loanhist) THEN COUNT(LH.isbn) + (
            SELECT COUNT(*)
            FROM member AS M1
            INNER JOIN loanhist AS LH1
            ON M1.member_no = LH1.member_no AND M1.member_no = M.member_no AND YEAR(LH1.out_date) = 2001
        )
        ELSE 0
    END
FROM juvenile AS J
INNER JOIN member AS M
ON M.member_no = J.member_no
INNER JOIN adult AS A
ON A.member_no = J.adult_member_no
INNER JOIN member AS MP
ON MP.member_no = J.adult_member_no
LEFT OUTER JOIN loanhist AS LH
ON LH.member_no = M.member_no AND YEAR(LH.out_date) = 2001 
GROUP BY M.member_no, M.firstname, M.lastname, A.street, A.city, A.state, MP.firstname, MP.lastname
ORDER BY 1

**3.3 Kategorie które w roku 1997 grudzień były obsłużone wyłącznie przez ‘United
Package**

In [ ]:
USE Northwind

In [ ]:
SELECT DISTINCT C.CategoryName
FROM Orders AS O
INNER JOIN Shippers AS S
ON O.ShipVia = S.ShipperID AND S.CompanyName = 'United Package'
INNER JOIN [Order Details] AS OD
ON OD.OrderID = O.OrderID
INNER JOIN Products AS P
ON P.ProductID = OD.ProductID
INNER JOIN Categories AS C
ON C.CategoryID = P.CategoryID
WHERE YEAR(O.ShippedDate) = 1997 AND MONTH(O.ShippedDate) = 12

**3.4 Wybierz klientów, którzy kupili przedmioty wyłącznie z jednej kategorii w marcu
1997 i wypisz nazwę tej kategorii**

In [ ]:
SELECT DISTINCT C.CustomerID,
    (SELECT CA1.CategoryName
    FROM Customers AS C1
    INNER JOIN Orders AS O1
    ON O1.CustomerID = C1.CustomerID AND YEAR(O1.OrderDate) = 1997 AND MONTH(O1.OrderDate) = 3
    INNER JOIN [Order Details] AS OD1
    ON OD1.OrderID = O1.OrderID
    INNER JOIN Products AS P1
    ON P1.ProductID = OD1.ProductID
    INNER JOIN Categories AS CA1
    ON CA1.CategoryID = P1.CategoryID
    WHERE C1.CustomerID = C.CustomerID) AS Category
FROM Customers AS C
INNER JOIN Orders AS O 
ON C.CustomerID = O.CustomerID AND YEAR(O.OrderDate) = 1997 AND MONTH(O.OrderDate) = 3
INNER JOIN [Order Details] AS OD
ON OD.OrderID = O.OrderID
INNER JOIN Products AS P
ON OD.ProductID = P.ProductID
GROUP BY C.CustomerID
HAVING COUNT(P.CategoryID) = 1

## Zadania #4
**4.1 Wybierz dzieci wraz z adresem, które nie wypożyczyły książek w lipcu 2001
autorstwa ‘Jane Austin’**

In [ ]:
USE Library

In [ ]:
SELECT M.firstname, M.lastname, A.street, A.city, A.state
FROM juvenile AS J
INNER JOIN member AS M
ON M.member_no = J.member_no
INNER JOIN adult AS A
ON A.member_no = J.adult_member_no
WHERE J.member_no NOT IN
    (SELECT J2.member_no
    FROM juvenile AS J2
    INNER JOIN loanhist AS LH2
    ON J2.member_no = LH2.member_no AND YEAR(LH2.out_date) = 2001 AND MONTH(LH2.out_date) = 7
    INNER JOIN title AS T2
    ON LH2.title_no = T2.title_no AND T2.author = 'Jane Austin')

**4.2 Wybierz kategorię, która w danym roku 1997 najwięcej zarobiła, podział na miesiące**

In [ ]:
USE Northwind

In [ ]:
SELECT TOP 1 P.CategoryID
FROM Products AS P
INNER JOIN [Order Details] AS OD
ON P.ProductID = OD.ProductID
GROUP BY P.CategoryID
ORDER BY SUM(OD.UnitPrice * OD.Quantity * (1-OD.Discount)) DESC

**4.3 Dane pracownika i najczęstszy dostawca pracowników bez podwładnych**

In [ ]:
WITH  EmpNoSub
AS (SELECT E.EmployeeID, E.FirstName, E.LastName
    FROM Employees AS E
    LEFT OUTER JOIN Employees AS E2
    ON E.EmployeeID = E2.ReportsTo
    WHERE E2.EmployeeID IS NULL),
tab
AS (SELECT ENS.EmployeeID, O.ShipVia, COUNT(*) as cnt
    FROM EmpNoSub AS ENS
    INNER JOIN Orders AS O
    ON ENS.EmployeeID = O.EmployeeID
    GROUP BY ENS.EmployeeID, O.ShipVia)

SELECT tab.EmployeeID, tab.ShipVia
FROM  tab
WHERE tab.cnt = 
    (SELECT MAX(tab2.cnt)
    FROM tab AS tab2
    WHERE tab2.EmployeeID = tab.EmployeeID)

**4.4 Wybierz tytuły książek, gdzie ilość wypożyczeń książki jest większa od średniej ilości
wypożyczeń książek tego samego autora.**

In [ ]:
USE Library

In [ ]:
--może być źle, nie uwzględnia loan

SELECT T.title, COUNT(*)
FROM title AS T
INNER JOIN loanhist AS LH
ON LH.title_no = T.title_no
GROUP BY T.title_no, T.title, T.author
HAVING COUNT(*) > 
    (SELECT AVG(tab.num)
    FROM
    (SELECT T.author, T.title_no, COUNT(*) AS num
    FROM title AS T
    INNER JOIN loanhist AS LH
    ON T.title_no = LH.title_no
    GROUP BY T.author, T.title_no) AS tab
    WHERE tab.author = T.author
    GROUP BY tab.author)